# Foil Stats

Copyright 2022 Michael George (AKA Logiqx).

This file is part of [GPS Wizard](https://github.com/Logiqx/gps-wizard) and is distributed under the terms of the GNU General Public License.

GPS Wizard is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

GPS Wizard is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with GPS Wizard. If not, see <https://www.gnu.org/licenses/>.

In [1]:
import os
import sys
import json

import numpy as np
import matplotlib.pyplot as plt

In [2]:
def processFoils():
    '''Process foils in JSON'''
    
    filename = os.path.join(projdir, 'config', 'foils.json')
    
    with open(filename) as f:
        foils = json.loads(f.read())
        
        for brandName, ranges in foils.items():
            for rangeName, rangeData in ranges.items():

                dataItems = []

                for wingName, wingData in rangeData['Front'].items():
                    if 'Span' in wingData and 'Area' in wingData:
                        dataItem = {}

                        dataItem['label'] = wingName
                        dataItem['span'] = wingData['Span']
                        dataItem['area'] = wingData['Area']
                        dataItem['spansq'] = wingData['Span'] * wingData['Span']
                        dataItem['aspect'] = (wingData['Span'] / 10) ** 2 / wingData['Area']
                        
                        dataItems.append(dataItem)

                if len(dataItems) > 1:
                    chartDef = {}

                    chartDef['brand'] = brandName
                    chartDef['range'] = rangeName

                    drawSpanChart(chartDef, dataItems)

                    
def drawSpanChart(chartDef, dataItems):
    '''Draw chart of span² vs area'''

    chartDef['xitem'] = 'area'
    chartDef['yitem'] = 'spansq'
    chartDef['xticks'] = np.arange(0, 1100, 100)
    chartDef['yticks'] = np.arange(0, 1100000, 100000)
    chartDef['xlabel'] = 'Area (cm²)'
    chartDef['ylabel'] = 'Span² (mm²)'

    drawLineChart(chartDef, dataItems)

                    
def drawAspectChart(chartDef, dataItems):
    '''Draw chart of aspect ratio vs area'''

    chartDef['xitem'] = 'area'
    chartDef['yitem'] = 'aspect'
    chartDef['xticks'] = np.arange(0, 1100, 100)
    chartDef['yticks'] = np.arange(0, 13, 1)
    chartDef['xlabel'] = 'Area (cm²)'
    chartDef['ylabel'] = 'Aspect Ratio'

    drawLineChart(chartDef, dataItems)


def drawLineChart(chartDef, dataItems):
    '''Draw line chart for single foil range'''

    fig, ax = plt.subplots(1, figsize=(8, 4), dpi=100)

    # Axis title and labels
    ax.set_title('{} - {}'.format(chartDef['brand'], chartDef['range']))
    ax.set_xlabel(chartDef['xlabel'])
    ax.set_ylabel(chartDef['ylabel'])  

    # Simple scatter chart
    x = [dataItem[chartDef['xitem']] for dataItem in dataItems]
    y = [dataItem[chartDef['yitem']] for dataItem in dataItems]
    ax.scatter(x, y)

    for i, dataItem in enumerate(dataItems):
        label = dataItem['label']
        ax.annotate(label, (x[i], y[i]), ha='right')

    # Limits for x and y, plus labels / ticks
    ax.set_xlim(chartDef['xticks'].min(), chartDef['xticks'].max())
    ax.set_ylim(chartDef['yticks'].min(), chartDef['yticks'].max())
    ax.set_xticks(chartDef['xticks'])
    ax.set_yticks(chartDef['yticks'])

    # Add trend line
    slope, intercept = np.polyfit(x, y, 1)
    x = np.linspace(chartDef['xticks'].min(), chartDef['xticks'].max(), chartDef['xticks'].max())
    y = slope * x + intercept
    ax.plot(x, y)

    # Save image
    filename = os.path.join(projdir, 'docs', 'img', chartDef['brand'], chartDef['range'] + '.png')
    dirname = os.path.dirname(filename)
    if not os.path.exists(dirname):
        os.makedirs(dirname)
    fig.savefig(filename, bbox_inches='tight', facecolor='w')

    # Close figure
    plt.close(fig)

In [3]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], ".."))

    processFoils()
    
    print(os.linesep + 'All done!')


All done!
